In [206]:
import torch
import torch.nn.functional as F
import torch.nn as nn

import cnn
import cnn_quantized as cnnq

from torch.utils.data import Dataset
from torchsummary import summary
import numpy as np
import matplotlib.pyplot as plt

import utils as ut
import pandas as pd
import cv2 as cv2
import os


dirname = os.getcwd()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [207]:
_, test_loader = cnn.dataset()

model = cnn.CNN().to(device)
quant_model = cnnq.CNN_Quantized(model).to(device)

state_dict = torch.load(f"{dirname}/weights/cnn_quant_weights.pth")
steps = torch.load(f"{dirname}/weights/cnn_quant_steps.pth")

for key in state_dict.keys():
    if "weight" in key:
        state_dict[key] = state_dict[key] * steps[key]
quant_model.load_state_dict(state_dict)
original_accuracy = cnn.test(quant_model, test_loader)

Test Accuracy: 93.74%


In [208]:
print("Before pruning")
cnn.test(quant_model, test_loader)
original_size = ut.get_model_size(quant_model)
print(original_size)
sparsity = {
    "conv1.0.weight": 0.12,
    "conv2.0.weight": 0.72,
    "conv3.0.weight": 0.75,
    "fc.1.weight": 0.77,
}

print("After pruning")
pruned_size, pruned_accuracy = quant_model.magnitude_prune(model, sparsity, test_loader)
print(pruned_size)

ratio = pruned_size / original_size * 100
print(f"Ratio: {float(ratio)}")

Before pruning
Test Accuracy: 93.36%
3362624
After pruning
Test Accuracy: 90.53%
tensor(772416, device='cuda:0')
Ratio: 22.970632553100586


In [ ]:
sparsity, accuracies = ut.sensitivity_scan(quant_model, test_loader, 0.04, 0.04, 1.0)

In [ ]:
ut.plot_num_parameters_distribution(quant_model)

In [ ]:
ut.plot_sensitivity_scan(quant_model, sparsity, accuracies, original_accuracy)